In [7]:
import pandas as pd
import warnings
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import snowballstemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import cross_val_score 
from sklearn import model_selection
from sklearn import naive_bayes
import string

warnings.simplefilter(action='ignore', category=FutureWarning)

df = pd.read_csv(r'D:\e-ticaret_urun_yorumlari.csv', delimiter=';')
df.head()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Metin,Durum
0,evet anlatıldığı gibi,1
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1
3,Daha öncede aynısını almıştım çok güzel ve kal...,1
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1


In [8]:
df.isna().value_counts()


Metin  Durum
False  False    15170
Name: count, dtype: int64

In [9]:
df["Durum"].value_counts()


Durum
0    6978
1    6799
2    1393
Name: count, dtype: int64

In [10]:
df['Yeni Metin'] = df['Metin']
#buyuk-kucuk donusumu
df['Yeni Metin'] = df['Yeni Metin'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df.head()

,Metin,Durum,Yeni Metin
0,evet anlatıldığı gibi,1,evet anlatıldığı gibi
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1,daha öncede almıştım bu cihazdan ense ve sakal...
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1,ürün gayet başarılı sakal kesmede başlık sayıs...
3,Daha öncede aynısını almıştım çok güzel ve kal...,1,daha öncede aynısını almıştım çok güzel ve kal...
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1,erkek kuaförüyüm ense ve sıfır sakal traşı içi...


In [11]:
df['Yeni Metin'] = df['Yeni Metin'].str.replace(r'[^\w\s]', '')
df.head()


,Metin,Durum,Yeni Metin
0,evet anlatıldığı gibi,1,evet anlatıldığı gibi
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1,daha öncede almıştım bu cihazdan ense ve sakal...
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1,ürün gayet başarılı sakal kesmede başlık sayıs...
3,Daha öncede aynısını almıştım çok güzel ve kal...,1,daha öncede aynısını almıştım çok güzel ve kal...
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1,erkek kuaförüyüm ense ve sıfır sakal traşı içi...


In [12]:
df['Yeni Metin'] = df['Yeni Metin'].str.replace(r'\d', '')
df.head()

,Metin,Durum,Yeni Metin
0,evet anlatıldığı gibi,1,evet anlatıldığı gibi
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1,daha öncede almıştım bu cihazdan ense ve sakal...
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1,ürün gayet başarılı sakal kesmede başlık sayıs...
3,Daha öncede aynısını almıştım çok güzel ve kal...,1,daha öncede aynısını almıştım çok güzel ve kal...
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1,erkek kuaförüyüm ense ve sıfır sakal traşı içi...


In [13]:
sw = stopwords.words('turkish')
df['Yeni Metin'] = df['Yeni Metin'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
df.head()

,Metin,Durum,Yeni Metin
0,evet anlatıldığı gibi,1,evet anlatıldığı
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1,öncede almıştım cihazdan ense sakal tüketmek o...
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1,ürün gayet başarılı sakal kesmede başlık sayıs...
3,Daha öncede aynısını almıştım çok güzel ve kal...,1,öncede aynısını almıştım güzel kaliteli bir ürün.
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1,erkek kuaförüyüm ense sıfır sakal traşı uygun ...


In [14]:
df[['Metin',"Yeni Metin"]][:10]

,Metin,Yeni Metin
0,evet anlatıldığı gibi,evet anlatıldığı
1,Daha öncede almıştım bu cihazdan ense ve sakal...,öncede almıştım cihazdan ense sakal tüketmek o...
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,ürün gayet başarılı sakal kesmede başlık sayıs...
3,Daha öncede aynısını almıştım çok güzel ve kal...,öncede aynısını almıştım güzel kaliteli bir ürün.
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,erkek kuaförüyüm ense sıfır sakal traşı uygun ...
5,ürün gerçekten çok güzel,ürün gerçekten güzel
6,Ürün beklediğimden güzel çıktı gayet kullanışl...,ürün beklediğimden güzel çıktı gayet kullanışl...
7,güzel makina tavsiye ederim,güzel makina tavsiye ederim
8,tavsiye edebileceğim çok güzel bir makina,tavsiye edebileceğim güzel bir makina
9,ürün geldiğinde şarjı vardı. ilk lullanım önce...,ürün geldiğinde şarjı vardı. ilk lullanım önce...


In [15]:
stemmer = snowballstemmer.stemmer('turkish')
df["Kök Metin"]=df["Yeni Metin"].apply(lambda x: " ".join([stemmer.stemWord(word) for word in x.split()]))
df.head()

,Metin,Durum,Yeni Metin,Kök Metin
0,evet anlatıldığı gibi,1,evet anlatıldığı,evet anlatıldık
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1,öncede almıştım cihazdan ense sakal tüketmek o...,önce almış cihaz en sakal tüketmek on numar sı...
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1,ürün gayet başarılı sakal kesmede başlık sayıs...,ür gayet başarıl sakal kesme başlık sayıs bira...
3,Daha öncede aynısını almıştım çok güzel ve kal...,1,öncede aynısını almıştım güzel kaliteli bir ürün.,önce aynı almış güzel kalitel bir ürün.
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1,erkek kuaförüyüm ense sıfır sakal traşı uygun ...,erkek kuaför en sıfır sakal traş uygu bir ür


In [16]:
tf = (df["Kök Metin"]).apply(lambda x: 
pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf

,index,0
0,evet,113.0
1,anlatıldık,66.0
2,önce,30.0
3,almış,270.0
4,cihaz,119.0
...,...,...
16345,ve.,1.0
16346,gwliyo.,1.0
16347,kapattigi,1.0
16348,ayari,1.0


In [17]:
vectorizer = TfidfVectorizer()
Xtf = vectorizer.fit_transform(df["Kök Metin"])

tokens = vectorizer.get_feature_names_out()

tf_idf = pd.DataFrame(data = Xtf.toarray(), columns = tokens)

print(tf_idf)

        00  0000  00000000   01   04  05468911163   0a  0dan  0la  0ldu  ...  \
0      0.0   0.0       0.0  0.0  0.0          0.0  0.0   0.0  0.0   0.0  ...   
1      0.0   0.0       0.0  0.0  0.0          0.0  0.0   0.0  0.0   0.0  ...   
2      0.0   0.0       0.0  0.0  0.0          0.0  0.0   0.0  0.0   0.0  ...   
3      0.0   0.0       0.0  0.0  0.0          0.0  0.0   0.0  0.0   0.0  ...   
4      0.0   0.0       0.0  0.0  0.0          0.0  0.0   0.0  0.0   0.0  ...   
...    ...   ...       ...  ...  ...          ...  ...   ...  ...   ...  ...   
15165  0.0   0.0       0.0  0.0  0.0          0.0  0.0   0.0  0.0   0.0  ...   
15166  0.0   0.0       0.0  0.0  0.0          0.0  0.0   0.0  0.0   0.0  ...   
15167  0.0   0.0       0.0  0.0  0.0          0.0  0.0   0.0  0.0   0.0  ...   
15168  0.0   0.0       0.0  0.0  0.0          0.0  0.0   0.0  0.0   0.0  ...   
15169  0.0   0.0       0.0  0.0  0.0          0.0  0.0   0.0  0.0   0.0  ...   

       şur   şö  şöför  şüküllü  şükür 

In [18]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(tf_idf,df["Durum"],test_size=0.30,random_state = 42 )

In [19]:
nb_multi = naive_bayes.MultinomialNB()
nb_model_multi = nb_multi.fit(train_x,train_y)
y_pred_nb_multi=nb_model_multi.predict(test_x)
print("Multinominal Accuracy:", accuracy_score(test_y, y_pred_nb_multi, normalize=True))
print(classification_report(test_y, y_pred_nb_multi))

print("***********************************************************")
y_pred_nb_multi_train=nb_model_multi.predict(train_x)
print("Multinominal train Accuracy:", accuracy_score(train_y, y_pred_nb_multi_train, normalize=True))
print(classification_report(train_y, y_pred_nb_multi_train))
print("***********************************************************")

print("test cross val skor:" , cross_val_score(nb_model_multi, test_x, test_y, cv = 10).mean())

print("train cross val skor:" ,cross_val_score(nb_model_multi, train_x, train_y, cv = 10).mean())

Multinominal Accuracy: 0.8626675455943749
              precision    recall  f1-score   support

           0       0.88      0.95      0.92      2065
           1       0.85      0.95      0.90      2051
           2       1.00      0.02      0.03       435

    accuracy                           0.86      4551
   macro avg       0.91      0.64      0.61      4551
weighted avg       0.88      0.86      0.82      4551

***********************************************************
Multinominal train Accuracy: 0.8853941049063
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      4913
           1       0.86      0.97      0.92      4748
           2       1.00      0.03      0.06       958

    accuracy                           0.89     10619
   macro avg       0.92      0.66      0.64     10619
weighted avg       0.90      0.89      0.85     10619

***********************************************************
test cross val skor: 0.85651243

In [ ]:
yorum = input("Analiz etmek istediğiniz yorumu girin lütfen: ")
yorum_lower = yorum.lower() 
yorum_punctuation_removed = ''.join([char for char in yorum_lower if char not in string.punctuation])  
yorum_stopwords_removed = ' '.join([word for word in yorum_punctuation_removed.split() if word not in sw]) 
yorum_stemmed = ' '.join([stemmer.stemWord(word) for word in yorum_stopwords_removed.split()])  

yorum_vectorized = vectorizer.transform([yorum_stemmed])

y_pred = nb_model_multi.predict(yorum_vectorized)


print("Yorum Sınıflandırma Sonucu:", y_pred[0])
#0 ->negatif
#1-> pozitif
#2->notr

